In [ ]:
"""
Web scrape for more info

1. speaker (birth year, place)
2. school

"""

In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import numpy as np
import re
import pickle

# import sys
# sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
# from loadpage import *
# import importlib
# from fictiondb import *

%load_ext autoreload
%autoreload 2
import importlib
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_4/py')
from google import *


In [111]:
%run -i '../py/google.py'

## 1. Load data

In [3]:
speech_df = pd.read_pickle('../dump/speech_df')
speech_df

,speaker,year,school,transcript,length
0,SIDDHARTHA MUKHERJEE,2018,Univ. of Southern California,I wish someone had told me at my own commenc...,14487
1,JESMYN WARD,2018,Tulane University,Persist. Be patient. Be well. Good morning....,14063
2,DAVID SEDARIS,2018,OBERLIN COLLEGE,"Thank you so much for having me, and for prese...",10587
3,NIGHT SHYAMALAN,2018,DREXEL UNIVERSITY,\nwhat's up dragons alright let's start\n\nwit...,26406
4,TERRY TEACHOUT,2018,HAMILTON HOLT SCHOOL,"I’m supposed to keep it short, and I approve o...",6288
...,...,...,...,...,...
436,WILLIAM ALLEN,1936,WHITE NORTHWESTERN UNIVERSITY,About all that a commencement orator can do fo...,14953
437,FRANKLIN D ROOSEVELT,1932,OGLETHORPE UNIVERSITY,"For me, as for you, this is a day of honorable...",16017
438,OPRAH WINFREY,1918,USC,Thank you Wallis Annenberg and a special thank...,15301
439,RALPH WALDO,1838,EMERSON HARVARD UNIVERSITY,"In this refulgent summer, it has been a luxury...",40403


## 2. Speaker info from Google
Scrape with Selenium

In [4]:
# Create list of speakers
speaker_list = list(speech_df.speaker)
len(speaker_list)

441

In [ ]:
# def google_speaker(speaker):
    
#     keyword = speaker
#     speaker_dict={}
#     headers = ["speaker","profession","birth","born","age","origin"]

#     # Selenium begins
#     driver.get("https://www.google.com")
#     search_bar = driver.find_element_by_xpath("//input[@name='q'][@type='text']")
#     search_bar.clear()
#     search_bar.send_keys(keyword)
#     #     print(speaker)
#     search_bar.send_keys(Keys.RETURN)
#     time.sleep(2)

#     # Set default value of result
#     profession, birth, born, age, origin = np.nan, np.nan, np.nan, np.nan, np.nan

#     # Find profession
#     try:
#         profession = driver.find_element_by_xpath("//div[@data-attrid='subtitle']").text
#     except:
#         pass

#     if profession is np.nan:
#         try:
#             description = driver.find_element_by_xpath("//div[@data-attrid='description']").text
#             profession = re.findall("(?<=was )(.*)(?= who)", description)[0]
#         except:
#             pass

#     # Find birth info    
#     try:
#         birth = driver.find_element_by_xpath("//div[@class='rVusze']").text
#     except:
#         pass

#     # Extract age and birth blace from birth info
#     # Age
#     try:
#         age = re.findall("age \d+", birth)[0].split(' ')[-1]
#     except:
#         pass

#     # Year born
#     try:
#         born = re.findall("Born: .* \d+", birth)[0].split(', ')[-1]
#     except:
#         pass

#     # Birth place
#     try:
#         origin = " ".join(birth.split(', ')[-2:])
#     except:
#         pass

#     speak_dict = dict(zip(headers,
#                 [speaker,profession,birth,born,age,origin]))

#     return speak_dict

In [159]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

# Open automated Chrome with fictionDB webpage
driver = webdriver.Chrome(chromedriver)

In [160]:
google_speaker("JAMES RYAN")

{'speaker': 'JAMES RYAN',
 'profession': 'South African actor',
 'birth': 'Born: October 18, 1952 (age 68 years), South Africa',
 'born': '1952 (age 68',
 'age': '68',
 'origin': '1952 (age 68 years) South Africa'}

In [5]:
# # Scrape
# results_speaker = []
# for speaker in speaker_list:
#     result = google_speaker(speaker)
#     results_speaker.append(result)

In [6]:
results_speaker

[{'speaker': 'SIDDHARTHA MUKHERJEE',
  'profession': 'American-Indian physician',
  'birth': 'Born: July 21, 1970 (age 50 years), New Delhi, India',
  'born': '1970 (age 50',
  'age': '50',
  'origin': 'New Delhi India'},
 {'speaker': 'JESMYN WARD',
  'profession': 'American novelist',
  'birth': 'Born: April 1, 1977 (age 43 years), De Lisle, MS',
  'born': '1977 (age 43',
  'age': '43',
  'origin': 'De Lisle MS'},
 {'speaker': 'DAVID SEDARIS',
  'profession': 'American humorist',
  'birth': 'Born: December 26, 1956 (age 64 years), Johnson City, NY',
  'born': '1956 (age 64',
  'age': '64',
  'origin': 'Johnson City NY'},
 {'speaker': 'NIGHT SHYAMALAN',
  'profession': 'American-Indian filmmaker',
  'birth': 'Born: August 6, 1970 (age 50 years), Mahé, India',
  'born': '1970 (age 50',
  'age': '50',
  'origin': 'Mahé India'},
 {'speaker': 'TERRY TEACHOUT',
  'profession': 'American author',
  'birth': 'Born: February 6, 1956 (age 65 years), Cape Girardeau, MO',
  'born': '1956 (age 65'

In [7]:
# Pickle scrape results
import pickle

with open('../dump/speaker_info_list.pkl', 'wb') as f:
    pickle.dump(results_speaker, f)

In [183]:
# Create DataFrame of speaker_info
speaker_info = pd.DataFrame(results_speaker)
speaker_info

,speaker,profession,birth,born,age,origin
0,SIDDHARTHA MUKHERJEE,American-Indian physician,"Born: July 21, 1970 (age 50 years), New Delhi,...",1970 (age 50,50,New Delhi India
1,JESMYN WARD,American novelist,"Born: April 1, 1977 (age 43 years), De Lisle, MS",1977 (age 43,43,De Lisle MS
2,DAVID SEDARIS,American humorist,"Born: December 26, 1956 (age 64 years), Johnso...",1956 (age 64,64,Johnson City NY
3,NIGHT SHYAMALAN,American-Indian filmmaker,"Born: August 6, 1970 (age 50 years), Mahé, India",1970 (age 50,50,Mahé India
4,TERRY TEACHOUT,American author,"Born: February 6, 1956 (age 65 years), Cape Gi...",1956 (age 65,65,Cape Girardeau MO
...,...,...,...,...,...,...
436,WILLIAM ALLEN,Attorney,"Born: August 5, 1704, Philadelphia, PA",1704,NaN,Philadelphia PA
437,FRANKLIN D ROOSEVELT,32nd U.S. President,"Born: January 30, 1882, Hyde Park, NY",1882,NaN,Hyde Park NY
438,OPRAH WINFREY,American host,"Born: January 29, 1954 (age 67 years), Koscius...",1954 (age 67,67,Kosciusko MS
439,RALPH WALDO,American essayist,"Born: May 25, 1803, Boston, MA",1803,NaN,Boston MA


In [184]:
# Clean up the born column
speaker_info['born'] = speaker_info['born'].apply(str).apply(lambda x: x.split(' ')[0])
speaker_info.drop(columns=['birth'],inplace=True)
speaker_info

,speaker,profession,born,age,origin
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India
1,JESMYN WARD,American novelist,1977,43,De Lisle MS
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY
3,NIGHT SHYAMALAN,American-Indian filmmaker,1970,50,Mahé India
4,TERRY TEACHOUT,American author,1956,65,Cape Girardeau MO
...,...,...,...,...,...
436,WILLIAM ALLEN,Attorney,1704,NaN,Philadelphia PA
437,FRANKLIN D ROOSEVELT,32nd U.S. President,1882,NaN,Hyde Park NY
438,OPRAH WINFREY,American host,1954,67,Kosciusko MS
439,RALPH WALDO,American essayist,1803,NaN,Boston MA


In [10]:
# Pickle
speaker_info.to_pickle('../dump/speaker_info')

In [185]:
# Check duplicates
speaker_info[speaker_info.duplicated(['speaker'])==True]

,speaker,profession,born,age,origin
32,SHERYL SANDBERG,Chief Operating Officer of Facebook,1969,51,Washington D.C.
52,CHIMAMANDA NGOZI ADICHIE,Writer,1977,43,Enugu Nigeria
70,MINDY KALING,American actress,1979,41,Cambridge MA
78,SHERYL SANDBERG,Chief Operating Officer of Facebook,1969,51,Washington D.C.
99,ED HELMS,American actor,1974,47,Atlanta GA
104,STEPHEN COLBERT,American comedian,1964,56,Washington D.C.
105,ED HELMS,American actor,1974,47,Atlanta GA
113,NEIL DEGRASSE,American astrophysicist,1958,62,New York NY
114,DAVID BROOKS,Commentator,1961,59,Toronto Canada
118,JOHN GREEN,American author,1977,43,Indianapolis IN


### Check weird info

#### A. Missing profession

In [186]:
speaker_info[speaker_info['profession'].isna()].shape

(42, 5)

In [187]:
# Identify speakers missing profession
missing_profession_df = speaker_info[speaker_info['profession'].isna()]

missing_profession_list = missing_profession_df['speaker'].tolist()
missing_profession_index = missing_profession_df.index.tolist()

In [188]:
# Fill in missing profession
update_profession = [
    'writer', 'actor', 'Educator', 'Professor', 'CEO of Right to Start', 
    'CEO of Microsoft', 'Professor', 'Poet', 'spiritual leader', 'Attorney',
    'Finance', 'Educator', 'Biologist', 'Tech (Cisco)', 'Environmental diplomat',
    'Pulitzer Prize winner author', 'Professor', 'Professor', 'Student', 'Tech (Instagram)',
    'spiritual (pastor)', 'Educator', 'Finance (investor)', 'Finance (investor)', 'Politician',
    'student', 'Culture (911 museum)', 'spiritual (pastor)', 'writer', 'writer',
    '??', 'Tech (Instagram)', 'Tech, educator', 'Tech, educator','Science (NASA)',
    'CEO (NanoHorizons)','writer', 'writer', 'CEO (Coca-cola)', 'CEO (Coca-cola)', 
    'student', 'Judge'
]

speaker_info_allp = speaker_info.copy()
for list_index, df_index in enumerate(missing_profession_index):
    speaker_info_allp['profession'].iloc[df_index] = update_profession[list_index]

In [191]:
# Check again if there's missing profession
speaker_info_allp[speaker_info_allp['profession'].isna()]

,speaker,profession,born,age,origin


In [192]:
# Pickle
speaker_info_allp.to_pickle('../dump/speaker_info_allp')

#### B. Missing born info

In [132]:
speaker_info[speaker_info.speaker == 'JOSE B. GONZALEZ']['born']

27    Born:
Name: born, dtype: object

In [134]:
wrong_born = speaker_info[speaker_info.born == "Born:"]
wrong_born

,speaker,profession,born,age,origin
27,JOSE B. GONZALEZ,Poet,Born:,54,San Salvador El Salvador
40,MARIA POPOVA,Bulgarian writer,Born:,36,Born: 1985 (age 36 years) Bulgaria
49,ARI WEINZWEIG,Businessperson,Born:,65,Born: 1956 (age 65 years)
56,JOHN SEELY,Researcher,Born:,81,Utica NY
67,IAN BRENNAN,American screenwriter,Born:,43,Mount Prospect IL
91,ZOKETSU NORMAN FISCHER,American poet,Born:,75,Wilkes-Barre PA
133,MAIRA KALMAN,American illustrator,Born:,72,Tel Aviv-Yafo Israel
178,STEVE BLANK,Entrepreneur,Born:,68,New York NY
197,DAVID COTE,American businessman,Born:,69,Manchester NH
198,MAKOTO FUJIMURA,Artist,Born:,61,Boston MA


In [149]:
wrong_born['born'] = 2021 - wrong_born['age'].astype(int)

<ipython-input-149-b8d4d5e8d249>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_born['born'] = 2021 - wrong_born['age'].astype(int)


In [150]:
wrong_born

,speaker,profession,born,age,origin
27,JOSE B. GONZALEZ,Poet,1967,54,San Salvador El Salvador
40,MARIA POPOVA,Bulgarian writer,1985,36,Born: 1985 (age 36 years) Bulgaria
49,ARI WEINZWEIG,Businessperson,1956,65,Born: 1956 (age 65 years)
56,JOHN SEELY,Researcher,1940,81,Utica NY
67,IAN BRENNAN,American screenwriter,1978,43,Mount Prospect IL
91,ZOKETSU NORMAN FISCHER,American poet,1946,75,Wilkes-Barre PA
133,MAIRA KALMAN,American illustrator,1949,72,Tel Aviv-Yafo Israel
178,STEVE BLANK,Entrepreneur,1953,68,New York NY
197,DAVID COTE,American businessman,1952,69,Manchester NH
198,MAKOTO FUJIMURA,Artist,1960,61,Boston MA


In [154]:
speaker_info[speaker_info.born == "Born:"]['born'] = 2021 - speaker_info[speaker_info.born == "Born:"]['age'].astype(int)

<ipython-input-154-55b07002dfa1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speaker_info[speaker_info.born == "Born:"]['born'] = 2021 - speaker_info[speaker_info.born == "Born:"]['age'].astype(int)


In [157]:
speaker_info[speaker_info.born == "Born:"]['born'] = wrong_born['born']

<ipython-input-157-ce7ff3a7a160>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speaker_info[speaker_info.born == "Born:"]['born'] = wrong_born['born']


In [158]:
speaker_info[speaker_info.born == "Born:"]

,speaker,profession,born,age,origin
27,JOSE B. GONZALEZ,Poet,Born:,54,San Salvador El Salvador
40,MARIA POPOVA,Bulgarian writer,Born:,36,Born: 1985 (age 36 years) Bulgaria
49,ARI WEINZWEIG,Businessperson,Born:,65,Born: 1956 (age 65 years)
56,JOHN SEELY,Researcher,Born:,81,Utica NY
67,IAN BRENNAN,American screenwriter,Born:,43,Mount Prospect IL
91,ZOKETSU NORMAN FISCHER,American poet,Born:,75,Wilkes-Barre PA
133,MAIRA KALMAN,American illustrator,Born:,72,Tel Aviv-Yafo Israel
178,STEVE BLANK,Entrepreneur,Born:,68,New York NY
197,DAVID COTE,American businessman,Born:,69,Manchester NH
198,MAKOTO FUJIMURA,Artist,Born:,61,Boston MA


## 3. School info from Google
Location (latitude, longitude)

In [12]:
# Create list of speakers
school_list = list(speech_df.school)
len(school_list)

441

In [13]:
def google_school(school):
    
    school = school
    keyword = school + ' coordinate dd'
    school_dict={}
    headers = ["school","coord","latitude","longitude"]

    # Selenium begins
    driver.get("https://www.google.com")
    search_bar = driver.find_element_by_xpath("//input[@name='q'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    #     print(speaker)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)

    # Set default value of result
    coord,latitude,longitude = np.nan,np.nan,np.nan

    # Find coordinate decimal degrees
    try:
        coord = driver.find_element_by_xpath("//span[@class='hgKElc']").text
    except:
        pass
    
    
    # Extract latitude and longitude
    try:
        latitude = re.findall('[-+]?\d*\.?[-+]?\d+', coord)[0]
        longitude = re.findall('[-+]?\d*\.?[-+]?\d+', coord)[1]
    except:
        pass

    school_dict = dict(zip(headers,
                [school,coord,latitude,longitude]))

    return school_dict

In [19]:
# Scrape
results_school = []
for school in school_list:
    result = google_school(school)
    results_school.append(result)

In [20]:
# Pickle scrape results
import pickle

with open('../dump/school_info_list.pkl', 'wb') as f:
    pickle.dump(results_school, f)

In [21]:
# Create DataFrame of speaker_info
school_info = pd.DataFrame(results_school)
school_info.drop(columns=['coord'],inplace=True)
school_info

,school,latitude,longitude
0,Univ. of Southern California,34.0205,-118.2856
1,Tulane University,29.9353,-90.1227
2,OBERLIN COLLEGE,41.2929,-82.2186
3,DREXEL UNIVERSITY,39.9564,-75.1887
4,HAMILTON HOLT SCHOOL,NaN,NaN
...,...,...,...
436,WHITE NORTHWESTERN UNIVERSITY,NaN,NaN
437,OGLETHORPE UNIVERSITY,NaN,NaN
438,USC,34.0205,-118.2856
439,EMERSON HARVARD UNIVERSITY,NaN,NaN


In [22]:
# Pickle
school_info.to_pickle('../dump/school_info')

In [58]:
# Examine missing values
missing_schools = school_info[school_info.latitude.isna()==True].school

In [59]:
len(missing_schools)

116

In [60]:
missing_schools

4               HAMILTON HOLT SCHOOL
6                           Darmouth
18               Agnes Scott College
24           Macaulay Honors College
25                              SCAD
                   ...              
435         CATT SWEET BRIAR COLLEGE
436    WHITE NORTHWESTERN UNIVERSITY
437            OGLETHORPE UNIVERSITY
439       EMERSON HARVARD UNIVERSITY
440                                 
Name: school, Length: 116, dtype: object

In [82]:
def google_school_2(school):
    """Get coordinates from google map url"""
    
    keyword = school
    school_dict={}
    headers = ["school","coord","latitude","longitude"]

    # Scrape google map for dd coord
    driver.get("https://www.google.com/maps")
    search_bar = driver.find_element_by_xpath("//input[@name='q'][@id='searchboxinput']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    #     print(speaker)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(5)

    coord,latitude,longitude = np.nan,np.nan,np.nan
    try:
        url = driver.current_url
        coord = re.findall('@[-+]?\d+\.?\d+,[-+]?\d+\.?\d+',url)
        latitude = coord[0].split(',')[0].lstrip('@')
        longitude = coord[0].split(',')[1]
    except:
        pass

    school_dict = dict(zip(headers,
                           [school,coord,latitude,longitude]))

    return school_dict

In [89]:
# Scrape
results_school_2 = []
for school in missing_schools:
    result = google_school_2(school)
    results_school_2.append(result)

In [90]:
# Pickle scrape results
import pickle

with open('../dump/school_info_list_2.pkl', 'wb') as f:
    pickle.dump(results_school_2, f)

In [12]:
import pickle
with open('../dump2/school_info_list_2.pkl', 'rb') as f:
    results_school_2 = pickle.load(f)

In [13]:
second_scrape = pd.DataFrame(results_school_2)

In [77]:
url = driver.current_url
url

'https://www.google.com/maps/place/Rollins+College+Hamilton+Holt+School/@28.5933647,-81.353002,17z/data=!3m1!4b1!4m5!3m4!1s0x88e7701ab5a84721:0x8e4f3b62db046783!8m2!3d28.5933647!4d-81.353002'

In [67]:
coord = re.findall('@[-+]?\d+\.?\d+,[-+]?\d+\.?\d+',url)

In [70]:
coord[0]

'@28.5933647,-81.353002'

In [72]:
latitude = coord[0].split(',')[0].lstrip('@')
longitude = coord[0].split(',')[1]
latitude, longitude

('28.5933647', '-81.353002')

### Let's step back and examine the situation

In [14]:
import pandas as pd
speech_df_speaker = pd.read_pickle('../dump/speech_df_speaker')
school_info = pd.read_pickle('../dump/school_info')

In [15]:
import pickle
with open('../dump2/school_info_list_2.pkl', 'rb') as f:
    results_school_2 = pickle.load(f)

In [16]:
second_scrape = pd.DataFrame(results_school_2)

In [17]:
speech_df_speaker

,speaker,profession,born,age,origin,category,year,school,transcript,length,age_then
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India,Med/Law,2018,Univ. of Southern California,I wish someone had told me at my own commenc...,14487,48
1,JESMYN WARD,American novelist,1977,43,De Lisle MS,Publish,2018,Tulane University,Persist. Be patient. Be well. Good morning....,14063,41
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY,Entertainment,2018,OBERLIN COLLEGE,"Thank you so much for having me, and for prese...",10587,62
3,NIGHT SHYAMALAN,American-Indian filmmaker,1970,50,Mahé India,Entertainment,2018,DREXEL UNIVERSITY,\nwhat's up dragons alright let's start\n\nwit...,26406,48
4,TERRY TEACHOUT,American author,1955,65,Cape Girardeau MO,Publish,2018,HAMILTON HOLT SCHOOL,"I’m supposed to keep it short, and I approve o...",6288,63
...,...,...,...,...,...,...,...,...,...,...,...
326,GUIDO CALABRESI,American-Italian judge,1932,88,Milan Italy,Med/Law,1988,CONNECTICUT COLLEGE,It's a particular honor for me to be here at t...,14101,56
327,MEREDITH MONK,American composer,1942,78,New York NY,Art,1985,SARAH LAWRENCE COLLEGE,"President Lichman, members of the faculty, alu...",9255,43
328,MARGARET ATWOOD,Canadian poet,1939,81,Ottawa Canada,Publish,1983,UNIVERSITY OF TORONTO,I am of course overjoyed to be here today in t...,11588,44
329,ALAN ALDA,American actor,1935,85,New York NY,Entertainment,1980,CONNECTICUT COLLEGE,I'm here today for a very special reason.\n\nW...,18161,45


In [18]:
school_info

,school,latitude,longitude
0,Univ. of Southern California,34.0205,-118.2856
1,Tulane University,29.9353,-90.1227
2,OBERLIN COLLEGE,41.2929,-82.2186
3,DREXEL UNIVERSITY,39.9564,-75.1887
4,HAMILTON HOLT SCHOOL,NaN,NaN
...,...,...,...
436,WHITE NORTHWESTERN UNIVERSITY,NaN,NaN
437,OGLETHORPE UNIVERSITY,NaN,NaN
438,USC,34.0205,-118.2856
439,EMERSON HARVARD UNIVERSITY,NaN,NaN


In [19]:
sss = pd.merge(speech_df_speaker,school_info,on='school',how='left').\
        drop_duplicates(subset=['speaker','year']).reset_index(drop=True)

In [20]:
sss

,speaker,profession,born,age,origin,category,year,school,transcript,length,age_then,latitude,longitude
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India,Med/Law,2018,Univ. of Southern California,I wish someone had told me at my own commenc...,14487,48,34.0205,-118.2856
1,JESMYN WARD,American novelist,1977,43,De Lisle MS,Publish,2018,Tulane University,Persist. Be patient. Be well. Good morning....,14063,41,29.9353,-90.1227
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY,Entertainment,2018,OBERLIN COLLEGE,"Thank you so much for having me, and for prese...",10587,62,41.2929,-82.2186
3,NIGHT SHYAMALAN,American-Indian filmmaker,1970,50,Mahé India,Entertainment,2018,DREXEL UNIVERSITY,\nwhat's up dragons alright let's start\n\nwit...,26406,48,39.9564,-75.1887
4,TERRY TEACHOUT,American author,1955,65,Cape Girardeau MO,Publish,2018,HAMILTON HOLT SCHOOL,"I’m supposed to keep it short, and I approve o...",6288,63,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,GUIDO CALABRESI,American-Italian judge,1932,88,Milan Italy,Med/Law,1988,CONNECTICUT COLLEGE,It's a particular honor for me to be here at t...,14101,56,41.3737,-72.1028
327,MEREDITH MONK,American composer,1942,78,New York NY,Art,1985,SARAH LAWRENCE COLLEGE,"President Lichman, members of the faculty, alu...",9255,43,NaN,NaN
328,MARGARET ATWOOD,Canadian poet,1939,81,Ottawa Canada,Publish,1983,UNIVERSITY OF TORONTO,I am of course overjoyed to be here today in t...,11588,44,43.6570,-79.3903
329,ALAN ALDA,American actor,1935,85,New York NY,Entertainment,1980,CONNECTICUT COLLEGE,I'm here today for a very special reason.\n\nW...,18161,45,41.3737,-72.1028


In [21]:
remove_missing = sss[~sss.isna().any(axis=1)]
remove_missing.shape

(259, 13)

In [22]:
missing_loc = sss[sss.isna().any(axis=1)]
missing_loc.shape

(72, 13)

In [23]:
fill_missing = pd.merge(missing_loc,second_scrape,on='school',how='left').\
                drop_duplicates(subset=['speaker','year']).reset_index(drop=True)
fill_missing.drop(columns=['latitude_x','longitude_x','coord'],inplace=True)
fill_missing.rename(columns = {'latitude_y':'latitude', 'longitude_y':'longitude'},inplace=True)
fill_missing

,speaker,profession,born,age,origin,category,year,school,transcript,length,age_then,latitude,longitude
0,TERRY TEACHOUT,American author,1955,65,Cape Girardeau MO,Publish,2018,HAMILTON HOLT SCHOOL,"I’m supposed to keep it short, and I approve o...",6288,63,28.5933647,-81.353002
1,TERRY TEACHOUT,American author,1955,65,Cape Girardeau MO,Publish,2011,Hamilton Holt School,"TRANSCRIPT\n Im supposed to keep it short, an...",6383,56,28.5933647,-81.353002
2,MINDY KALING,American actress,1979,41,Cambridge MA,Entertainment,2018,Darmouth,"Good morning to the Class of 2018, the facul...",15391,39,43.7052043,-72.2894965
3,MINDY KALING,American actress,1979,41,Cambridge MA,Entertainment,2014,HARVARD LAW SCHOOL,"Graduates, parents, faculty, this is really su...",12739,35,42.3780901,-71.1188478
4,OPRAH WINFREY,American host,1953,67,Kosciusko MS,Entertainment,2017,Agnes Scott College,You're nothing if you are not the truth. Ex...,1264,64,33.7685056,-84.2945346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,CARLY FIORINA,American businesswoman,1954,66,Austin TX,Tech/Business,2000,MIT,I'd like to start my remarks this morning with...,17740,46,42.360091,-71.09416
68,CALLIE KHOURI,American television screenwriter,1957,63,San Antonio TX,Publish,1994,SWEET BRIAR COLLEGE,"I'm so happy to be here today. Now, I don't wa...",19940,37,37.5576478,-79.0828644
69,NAOMI WOLF,American author,1962,58,San Francisco CA,Publish,1992,SCRIPPS COLLEGE,Even the best of revolutions can go awry when ...,7773,30,34.1039266,-117.7101745
70,MEREDITH MONK,American composer,1942,78,New York NY,Art,1985,SARAH LAWRENCE COLLEGE,"President Lichman, members of the faculty, alu...",9255,43,40.93459,-73.8455821


In [24]:
ssss = pd.concat([remove_missing,fill_missing],axis=0).reset_index(drop=True)

In [25]:
ssss

,speaker,profession,born,age,origin,category,year,school,transcript,length,age_then,latitude,longitude
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India,Med/Law,2018,Univ. of Southern California,I wish someone had told me at my own commenc...,14487,48,34.0205,-118.2856
1,JESMYN WARD,American novelist,1977,43,De Lisle MS,Publish,2018,Tulane University,Persist. Be patient. Be well. Good morning....,14063,41,29.9353,-90.1227
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY,Entertainment,2018,OBERLIN COLLEGE,"Thank you so much for having me, and for prese...",10587,62,41.2929,-82.2186
3,NIGHT SHYAMALAN,American-Indian filmmaker,1970,50,Mahé India,Entertainment,2018,DREXEL UNIVERSITY,\nwhat's up dragons alright let's start\n\nwit...,26406,48,39.9564,-75.1887
4,ABBY WAMBACK,American soccer player,1980,40,Rochester NY,Sport,2018,Barnard College,"Failure is not something to be ashamed of, ...",15866,38,40.8096,-73.9635
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,CARLY FIORINA,American businesswoman,1954,66,Austin TX,Tech/Business,2000,MIT,I'd like to start my remarks this morning with...,17740,46,42.360091,-71.09416
327,CALLIE KHOURI,American television screenwriter,1957,63,San Antonio TX,Publish,1994,SWEET BRIAR COLLEGE,"I'm so happy to be here today. Now, I don't wa...",19940,37,37.5576478,-79.0828644
328,NAOMI WOLF,American author,1962,58,San Francisco CA,Publish,1992,SCRIPPS COLLEGE,Even the best of revolutions can go awry when ...,7773,30,34.1039266,-117.7101745
329,MEREDITH MONK,American composer,1942,78,New York NY,Art,1985,SARAH LAWRENCE COLLEGE,"President Lichman, members of the faculty, alu...",9255,43,40.93459,-73.8455821


In [26]:
# Check again for any missing values
ssss[ssss.isna().any(axis=1)]

,speaker,profession,born,age,origin,category,year,school,transcript,length,age_then,latitude,longitude


In [27]:
# Pickle

ssss.to_pickle('../dump/speech_speaker_school')

## Testing section

In [32]:
url = driver.current_url
url

'https://www.google.com/maps/place/Agnes+Scott+College/@33.7685056,-84.2967233,17z/data=!4m12!1m6!3m5!1s0x88f50739ac7ef53b:0xd2d77073183e1d81!2sAgnes+Scott+College!8m2!3d33.7685056!4d-84.2945346!3m4!1s0x88f50739ac7ef53b:0xd2d77073183e1d81!8m2!3d33.7685056!4d-84.2945346'

In [52]:
re.findall('@[-+]?\d+\.?\d+,[-+]?\d+\.?\d+,',url)

['@33.7685056,-84.2967233,']

In [ ]:
'[-+]?\d*\.?[-+]?\d+'

In [94]:
def google_school_test(school):
    
    school = school
    keyword = school + ' coordinate dd'
    school_dict={}
    headers = ["school","coord","latitude","longitude"]

    # Selenium begins
    driver.get("https://www.google.com")
    search_bar = driver.find_element_by_xpath("//input[@name='q'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    #     print(speaker)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)

    # Set default value of result
    coord,latitude,longitude = np.nan,np.nan,np.nan

    # Find coordinate decimal degrees
    try:
        coord = driver.find_element_by_xpath("//span[@class='hgKElc']").text
    except:
        pass
    
    
    # Extract latitude and longitude
    try:
        latitude = re.findall('[-+]?\d*\.?[-+]?\d+', coord)[0]
        longitude = re.findall('[-+]?\d*\.?[-+]?\d+', coord)[1]
    except:
        pass
    
    
    if coord is np.nan:
        school_dict = google_school_2(school)
    
    else:
        school_dict = dict(zip(headers,
                [school,coord,latitude,longitude]))

    return school_dict

In [97]:
# Scrape
results_school_3 = []
for school in school_list:
    result = google_school_test(school)
    results_school_3.append(result)

In [102]:
second = pd.DataFrame(results_school_3)

In [99]:
# # Pickle scrape results
# import pickle

# with open('../dump/school_coord_list.pkl', 'wb') as f:
#     pickle.dump(results_school_3, f)

In [103]:
second

,school,coord,latitude,longitude
0,Univ. of Southern California,Latitude: 34.0205 Longitude: -118.2856.,34.0205,-118.2856
1,Tulane University,Latitude: 29.9353 Longitude: -90.1227.,29.9353,-90.1227
2,OBERLIN COLLEGE,Latitude: 41.2929 Longitude: -82.2186.,41.2929,-82.2186
3,DREXEL UNIVERSITY,Latitude: 39.9564 Longitude: -75.1887.,39.9564,-75.1887
4,HAMILTON HOLT SCHOOL,"[@28.5933647,-81.353002]",28.5933647,-81.353002
...,...,...,...,...
436,WHITE NORTHWESTERN UNIVERSITY,"[@34.422784,-119.6938525]",34.422784,-119.6938525
437,OGLETHORPE UNIVERSITY,"[@34.422784,-119.6938525]",34.422784,-119.6938525
438,USC,Latitude: 34.0205 Longitude: -118.2856.,34.0205,-118.2856
439,EMERSON HARVARD UNIVERSITY,"[@34.422784,-119.6938525]",34.422784,-119.6938525


In [106]:
second[second.latitude == '34.422784'].count()

school       46
coord        46
latitude     46
longitude    46
dtype: int64

In [113]:
google_school_test('WHITE NORTHWESTERN UNIVERSITY')

{'school': 'WHITE NORTHWESTERN UNIVERSITY',
 'coord': ['@42.0564594,-87.6774557'],
 'latitude': '42.0564594',
 'longitude': '-87.6774557'}

In [119]:
# Scrape
results_school_4 = []
for school in school_list:
    result = google_school_2(school)
    results_school_4.append(result)

In [120]:
second = pd.DataFrame(results_school_4)

In [121]:
# Pickle scrape results
import pickle

with open('../dump/school_coord_list_2.pkl', 'wb') as f:
    pickle.dump(results_school_4, f)

In [122]:
second

,school,coord,latitude,longitude
0,Univ. of Southern California,"[@34.0413941,-118.2789549]",34.0413941,-118.2789549
1,Tulane University,"[@29.9407282,-90.1225054]",29.9407282,-90.1225054
2,OBERLIN COLLEGE,"[@41.2958696,-82.2232777]",41.2958696,-82.2232777
3,DREXEL UNIVERSITY,"[@39.9566127,-75.1921328]",39.9566127,-75.1921328
4,HAMILTON HOLT SCHOOL,"[@28.5933647,-81.3551907]",28.5933647,-81.3551907
...,...,...,...,...
436,WHITE NORTHWESTERN UNIVERSITY,"[@42.0564594,-87.6774557]",42.0564594,-87.6774557
437,OGLETHORPE UNIVERSITY,"[@33.8761916,-84.3369015]",33.8761916,-84.3369015
438,USC,"[@34.0223519,-118.2873057]",34.0223519,-118.2873057
439,EMERSON HARVARD UNIVERSITY,"[@34.4227029,-119.8164285]",34.4227029,-119.8164285


In [123]:
second[second.latitude=='34.4227029']

,school,coord,latitude,longitude
439,EMERSON HARVARD UNIVERSITY,"[@34.4227029,-119.8164285]",34.4227029,-119.8164285


In [124]:
second.to_pickle('../dump/school_coord_df')